# 大模型API

- 云端大模型：https://bailian.console.aliyun.com/
- 本地大模型：https://qwen.readthedocs.io/zh-cn/latest/getting_started/quickstart.html

In [23]:
import os
from openai import OpenAI


client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    # https://bailian.console.aliyun.com/?tab=model#/api-key
api_key="sk-dvzrirhbryogkhcnrxoauhmkwlztphvihcishgvkciofjvsx", 
    base_url="https://api.siliconflow.cn/v1/"
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="Qwen/QwQ-32B", # 这个模型免费
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"},
    ],
    # Qwen3模型通过enable_thinking参数控制思考过程（开源版默认True，商业版默认False）
    # 使用Qwen3开源版模型时，若未启用流式输出，请将下行取消注释，否则会报错
    #  
)
print(completion.model_dump_json())

{"id":"019772ff5438f8ef868a390cfeb671b0","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"\n\n你好！我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字（如写故事、写公文、写邮件、写剧本等），还能进行逻辑推理、编程，甚至表达观点和玩游戏。\n\n你可以把我当成一个全能型的智能助手，无论是学术问题、生活常识、科技知识，还是创意写作，我都会尽力为你提供帮助。如果你有任何问题或需要协助的地方，随时告诉我！😊","refusal":null,"role":"assistant","annotations":null,"audio":null,"function_call":null,"tool_calls":null,"reasoning_content":"嗯，用户问我“你是谁？”，我需要先确定他们想了解的具体内容。他们可能想知道我的身份、功能或者我能提供什么帮助。首先，我应该简要介绍自己作为阿里巴巴集团旗下的AI助手，然后说明我的主要功能，比如回答问题、提供建议、协助创作等。可能还需要提到我的语言能力和支持的领域，比如科技、文化、生活等方面。另外，用户可能希望知道如何与我互动，或者有什么特别之处，比如多语言支持或实时信息处理能力。我需要保持回答简洁明了，同时涵盖关键点，避免信息过载。另外，要确保用口语化的中文，让回答更自然。有没有需要特别强调的部分？比如我的训练数据截止时间或者隐私政策？不过用户可能更关心我能直接提供什么帮助。可能需要以开放式的结尾，邀请他们提出具体问题。检查是否有遗漏的重要信息，比如我的名字通义千问，以及所属的模型系列。确保回答友好且有帮助，让用户感到欢迎。\n"}}],"created":1749981025,"model":"Qwen/QwQ-32B","object":"chat.completion","service_tier":null,"system_fingerprint":"","usage":{"completion_tokens":310,"prompt_tokens":24,"total_tokens":334,"completion_tok

In [24]:
completion.choices[0].message.content

'\n\n你好！我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我可以帮助你回答问题、创作文字（如写故事、写公文、写邮件、写剧本等），还能进行逻辑推理、编程，甚至表达观点和玩游戏。\n\n你可以把我当成一个全能型的智能助手，无论是学术问题、生活常识、科技知识，还是创意写作，我都会尽力为你提供帮助。如果你有任何问题或需要协助的地方，随时告诉我！😊'

In [25]:
import pandas as pd
import glob, json, re
import numpy as np

In [26]:
pd.read_json("样例集/data.jsonl", lines=True)

,material_id,rule_id,rule,result
0,m_00001s,r_00001,该产品的保障责任在各材料中的定义没有冲突,False
1,m_00002s,r_00002,该产品的责任免除在各材料中的定义没有冲突,False
2,m_00003s,r_00003,该产品的责任免除在各材料中的定义没有冲突,False
3,m_00004s,r_00004,该产品的保障相关时间在各材料中的定义没有冲突,True
4,m_00005s,r_00005,该产品的赔付 & 领取规则在各材料中的定义没有冲突,False
5,m_00006s,r_00006,该产品的保障相关时间在各材料中的定义没有冲突,True
6,m_00007s,r_00007,该产品的投保条款在各材料中的定义没有冲突,False
7,m_00008s,r_00008,该产品的赔付 & 领取规则在各材料中的定义没有冲突,True
8,m_00009s,r_00009,该产品的基础产品销售信息在各材料中的定义没有冲突,False
9,m_00010s,r_00010,该产品的投保条款在各材料中的定义没有冲突,True


In [27]:
pd.read_json("样例集/data.jsonl", lines=True)["rule"].value_counts()

rule
该产品的保障责任在各材料中的定义没有冲突         4
该产品的责任免除在各材料中的定义没有冲突         4
该产品的赔付 & 领取规则在各材料中的定义没有冲突    3
该产品的投保条款在各材料中的定义没有冲突         3
该产品的保障相关时间在各材料中的定义没有冲突       2
该产品的基础产品销售信息在各材料中的定义没有冲突     2
该产品的附加条款在各材料中的定义没有冲突         1
该产品的与保障相关的时间在各材料中的定义没有冲突     1
Name: count, dtype: int64

In [28]:
pd.read_json("测试 A 集/data.jsonl", lines=True)["rule"].value_counts()

rule
该产品的赔付 & 领取规则在各材料中的定义没有冲突    29
该产品的责任免除在各材料中的定义没有冲突         28
该产品的投保条款在各材料中的定义没有冲突         28
该产品的保障责任在各材料中的定义没有冲突         21
该产品的基础产品销售信息在各材料中的定义没有冲突     14
该产品的退保条款在各材料中的定义没有冲突         13
该产品的附加条款在各材料中的定义没有冲突         13
该产品的续保条款在各材料中的定义没有冲突         11
该产品的与保障相关的时间在各材料中的定义没有冲突     11
该产品的出险条款在各材料中的定义没有冲突         11
该产品的术语解释在各材料中的定义没有冲突         11
该产品的保障相关时间在各材料中的定义没有冲突       10
Name: count, dtype: int64

# 方案1：基础提示词思路

In [29]:
def get_chunk_list(lst, chunk_size=200):
    """Yield successive chunk_size chunks from lst."""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

def exteract_module_raw_text(module_name, lines):
    completion = client.chat.completions.create(
        model="Qwen/QwQ-32B",
        messages=[
            {"role": "system", "content": f"""你是一个专业的保险行业的信息处理专家，请对下面的文本中抽取出与{module_name}相关的文本，只需要输出原文，不要有其他输出。如果相关的本文本不存在，则输出一个空格。"""},
            {"role": "user", "content": "".join(lines)},
        ],
         
    )
    return completion.choices[0].message.content

def analysis_conflict(module_name, content1, content2):
    completion = client.chat.completions.create(
        model="Qwen/QwQ-32B",
        messages=[
            {"role": "system", "content": f"""你是一个专业的保险行业的信息处理专家，请对下面文本进行严谨的一致性进行分析，判断是否相同条件的表达存在不一致的情况，只需要回答一致或不一致，不要有其他任何输出。
            
- 基础产品销售信息：该保险产品的基础配置信息，包括产品名、附加的条款信息、销售限制等；
- 投保条款：投保过程中的缴费约定、投被保人条件限制等；
- 保障责任：约定该产品的保险责任细节，如保障范围、保险金额、增值服务等；
- 保障相关时间：约定该产品的各类时间信息，包括但不限于犹豫期、等待期、宽限期等；
- 赔付 & 领取规则：约定该产品的保险责任的赔付、给付、领取及免赔细节，如赔付年龄/比例/次数等；
- 责任免除：约定该产品不承担保险责任的情形；
- 续保条款：约定续保相关信息，包括但不限于续保条件、保证续保等；
- 退保条款：约定退保相关信息，包括但不限于退保条件、退保手续费等；
- 出险条款：约定出险相关信息，包括但不限于出险地点、出险方式等；
- 附加条款：约定该产品的附加条款，如特别约定等；
- 术语解释：约定该产品的术语解释，如名词定义等；"""},
            {"role": "user", "content": f"""片段1\n{content1}\n\n片段2\n{content2}\n\n分析上述两个片段的{module_name}是否存在相同的条件但表达不一致的情况。"""},
        ],
         
    )
    return completion.choices[0].message.content

In [31]:
# 在导入部分添加
from tqdm.notebook import tqdm

# 修改主循环部分
for row in tqdm(pd.read_json("样例集/data.jsonl", lines=True).iloc[2:4].iterrows(), desc="处理数据"):
    module_name = row[1].rule.replace("该产品的", "").replace("在各材料中的定义没有冲突", "")

    module_content_list = []
    for path in glob.glob(f"样例集/materials/{row[1].material_id}/*/*"):
        lines = open(path).readlines()

        module_lines = ""
        for chunk_lines in get_chunk_list(lines):
            res = exteract_module_raw_text(module_name, chunk_lines)
            if res:
                module_lines = module_lines + res
        
        module_content_list.append(module_lines)

    print("\n")
    print(row[1].material_id)
    for i in range(len(module_content_list)):
        for j in range(i, len(module_content_list)):
            print(analysis_conflict(module_name, module_content_list[i], module_content_list[j]))

处理数据: 0it [00:00, ?it/s]

In [ ]:
test = pd.read_json("测试 A 集/data.jsonl", lines=True)
test

In [ ]:
pd.read_json("测试 A 集/data.jsonl", lines=True).iloc[124:]

In [ ]:
for row in pd.read_json("测试 A 集/data.jsonl", lines=True).iloc[:].iterrows():
    module_name = row[1].rule.replace("该产品的", "").replace("在各材料中的定义没有冲突", "")

    print("")
    print(row[1].material_id)

    try:
        module_content_list = []
        for path in glob.glob(f"测试 A 集/materials/{row[1].material_id}/*/*"):
            print(path)
            lines = open(path).readlines()
            module_lines = ""
            for chunk_lines in get_chunk_list(lines):
                res = exteract_module_raw_text(module_name, chunk_lines)
                if res:
                    module_lines = module_lines + res
            
            module_content_list.append(module_lines)
    
        result = [0, 1]
        for i in range(len(module_content_list)):
            for j in range(i, len(module_content_list)):
                if "不一致" in analysis_conflict(module_name, module_content_list[i], module_content_list[j]):
                    result.append(1)
                else:
                    result.append(0)
    
        with open("submit.jsonl", "a") as up:
            up.write(json.dumps({
                "material_id": row[1].material_id,
                "rule_id": row[1].rule_id,
                "result": bool(np.mean(result) < 0.3)
            }) + "\n")
    
        print(np.mean(result))
    except:
        with open("submit.jsonl", "a") as up:
            up.write(json.dumps({
                "material_id": row[1].material_id,
                "rule_id": row[1].rule_id,
                "result": False
            }) + "\n")